In [1]:
import os
import re
from pathlib import Path
from dotenv import load_dotenv

from langchain_groq import ChatGroq

# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated # to construct the agent's state
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add] # operator.add allows us to add messages instead of replacing them when the LLM's output is returned

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState) # initialize a stateful graph
        memory = MemorySaver() # initialize

        graph.add_node("llm", self.call_groq) # Add LLM node
        graph.add_node("action", self.take_action) # Add Tool node
        
        # The edge where the decision to use a tool is made
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        
        # Create edge and set starting point of the graph
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        self.graph = graph.compile(checkpointer=memory) # Build graph
        
        self.tools = {t.name: t for t in tools} # Save the tools' names that can be used
        self.model = model.bind_tools(tools) # Provide the name of the tools to the agent

    # Used by the Agent to check if an action exists by checking the last message in the state which is supposed to contain the tool's info
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    # Get the LLM's response and update the Agent's State by adding the response to the messages
    def call_groq(self, state: AgentState):
        messages = state['messages']
        if self.system: messages = [SystemMessage(content=self.system)] + messages
        
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # Search for the tool and use it
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            
            # If tool not found
            if not t['name'] in self.tools: 
                print("\n ....tool name not found in list of tools....")
                result = "bad tool name, retry"  # instruct LLM to retry
            
            # If tool exists, use it
            else:
                result = self.tools[t['name']].invoke(t['args']) 

            # Save the message returned from the tool
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

        print("Back to the model!")
        return {'messages': results}

In [2]:
# Create the tool to be used by the Agent
from langchain.graphs import Neo4jGraph
from langchain_core.tools import tool

# Connect to graph
graph = Neo4jGraph()

@tool
def query_graph(query):
  """Requires get_graph_schema to be run before using this function. This function is to Query from Neo4j knowledge graph using Cypher."""
  return graph.query(query)

In [ ]:
task = "Task: You are a career guide. Your job is to ask me up to 3 questions to uncover my personality traits according to the RAISEC model. You will ask these questions in a conversational flow where you will ask the second question after I answer the first. Once you understand my personality, you will stop asking questions and use a Neo4j database to improve your knowledge on compatible career paths for me. You will query the possible occupation titles that are suitable for my character. At any point, I can ask you questions and you will answer normally, then you will continue your personality test."

goal = "Understand my personality and then suggest suitable career paths. Note: when asking your questions, please number them to keep track of the number of questions asked."

schema_context = f"Here is the graph's schema: {graph.structured_schema}."

property_values = f"Property Values: empty"

query_approach = "Querying approach: You will not use 'LIMIT'. If Property Values: empty, you will not use general queries and will not include 'WHERE' or try to specify property values inside your Cypher code."

output = "Your final output: Interpret all the queried data, choose up to 3 suitable careers for me, list them in bullet points and include a brief explanation of how each path suites my personality. Include Cypher code in your answer."

tone = "Output's tone: Make your output friendly, fun and easy to read."

personal_info = "Personal Info: I love people and I am a good listener. I enjoy observation and analysis. I prefer being with abults rather than with kids and I also have computer programming skills."

reminder = "Reminder: If Property Values: empty, you will not use 'WHERE' or try to specify property values inside your Cypher code. Under no circumstances should you use 'DELETE'. Find the occupations that suite my character."

prompt = f"{task}\ {goal}\ {schema_context}\ {property_values}\ {query_approach}\ {output}\ {tone}\ {personal_info}\ {reminder}"

In [ ]:
# Prepare agent and the initial prompt
model = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")
abot = Agent(model, [query_graph], system=prompt)

# Prepare Human message
query = "What will we do today"

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content=query)
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="I enjoy a balance between creative work, analysis, and teamwork.")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Reserved listener who observes")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="When problem solving I mostly use intuition with some logic")
for event in abot.graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
# ## Visualize Graph
from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())
Image(abot.graph.get_graph().draw_mermaid_png(), width=300)
# print(abot.graph.get_graph().draw_ascii())
